## Todo

ArrayListe als Cython/Numpy erstellen:
mit 5 Listen

position, gesch, masse, pos_neu, geschw_neu



methoden als nogil funktionen umschreiben







queue implementieren

multiprocessor code schreiben

worker pool nutzen

In [1]:
#Beispiel Listen zum Testen

python_position = [[149_597_890_000, 0,0], [0,0,0]]
python_geschwindigkeit = [[0, 29.786 * 10**3,0], [0,0,0]]
python_masse = [[5.97219 * 10 ** 24], [1.9889 * 10 ** 30]]
python_position_neu = []
python_geschwindigkeit_neu = []
python_position_list = []

In [2]:
%load_ext Cython

In [3]:
#Umwandlung aus Python Listen in Numpy Arrays

import numpy as np
position = np.array(python_position, dtype=np.float32)
geschwindigkeit = np.array(python_geschwindigkeit,dtype=np.float32)
masse = np.array(python_masse,dtype=np.float32)
position_neu = np.array(python_position_neu ,dtype=np.float32)
geschwindigkeit_neu = np.ndarray(python_geschwindigkeit_neu ,dtype=np.float32)

## Einzelne Cython Module

In [4]:
import os
os.mkdir('/tmp/cython_galaxy')

FileExistsError: [Errno 17] File exists: '/tmp/cython_galaxy'

In [90]:
%%file /tmp/cython_galaxy/abstand.pyx
cimport cython
#Abstands Funktion
#Abstand_Sonne = position[1] - position[0]

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (double [:, :] position_view , double[:] result ,int first, int second) nogil:   
    result[0] = position_view[second][0] - position_view[first][0]
    result[1] = position_view[second][1] - position_view[first][1]
    result[2] = position_view[second][2] - position_view[first][2]

Overwriting /tmp/cython_galaxy/abstand.pyx


In [91]:
%%file /tmp/cython_galaxy/betrag.pyx
cimport cython
from libc.math cimport sqrt


#Betrags Funktion
#Betrag_Erde  = sqrt( Abstand_Erde**2 + Abstand_Erde**2 + Abstand_Erde**2)
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Betrag (double [:] abstand_view , double[:] result) nogil:
    result[0] = sqrt(abstand_view[0]**2 + abstand_view[1]**2 + abstand_view[2]**2)

Overwriting /tmp/cython_galaxy/betrag.pyx


## Extrem komisches Verhalten

In [92]:
%%cython

#6.671999931335449
#10.0
#-11.0
#6.671999758234293e-11
#0.0
#WTF?????
cdef float g1 = 6.672
cdef float g2 = 10
cdef float g3 = -11
cdef float g = g1 * g2 ** g3
cdef float G = 6.672 * 10 ** -11

print(g1)
print(g2)
print(g3)
print(g)
print(G)

In [93]:
%%cython

#('G', 6.671999758234293e-11)
#('betrag', 3.347930846024836e+33)
#('masse_view[first_planet]', 5.972189887017193e+24)
#('masse_view[second_planet]', 1.9888999754506858e+30)
#('abstand_view[0]', 149597896704.0)
#('abstand_view[1]', 0.0)
#('abstand_view[2]', 0.0)
#('result[0]', inf)
#('result[0]', inf)
#('result[0]', inf)

#result[1] = G * (masse_view[first_planet] * masse_view[second_planet] / betrag) * abstand_view[1]
#result[2] = G * (masse_view[first_planet] * masse_view[second_planet] / betrag) * abstand_view[2]
cdef float g1 = 6.672
cdef float g2 = 10
cdef float g3 = -11

cdef float G = g1 * g2 ** g3
cdef float betrag = 3.347930846024836e+33
cdef float result

result = G * (5.972189887017193e+24 * 1.9888999754506858e+30 / betrag) * 149597896704.0
print(result)

In [94]:
%%file /tmp/cython_galaxy/kraft.pyx
cimport cython

#Kraft Funktion
#Kraft_Erde = G * (masse_erde * masse_sonne / (Betrag_Erde  ** 3)) * Abstand_Erde
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Kraft (double [:] masse_view, double [:] abstand_view,  double [:] Betrag, \
                  int first_planet, int second_planet, double[:] result) nogil:
    
    cdef double g1 = 6.672
    cdef double g2 = 10
    cdef double g3 = -11
    
    cdef double G = g1 * g2 ** g3
    cdef double betrag = Betrag[0] ** 3

    cdef double masse_multyply = masse_view[first_planet] * masse_view[second_planet]
    
    result[0] = G * (masse_multyply / betrag) * abstand_view[0]
    result[1] = G * (masse_multyply / betrag) * abstand_view[1]
    result[2] = G * (masse_multyply / betrag) * abstand_view[2]

Overwriting /tmp/cython_galaxy/kraft.pyx


In [95]:
%%file /tmp/cython_galaxy/beschleunigung.pyx
cimport cython

#Beschleunigungs Funktion
#Beschleunigung_Erde  = Kraft_Erde  / masse
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Beschleunigung (double [:] kraft_view , double [:] masse_view, int massen_index, double[:] result) nogil:
    cdef float G = 6.672 * 10 ** -11
    result[0] = kraft_view[0]  / masse_view[massen_index]
    result[1] = kraft_view[1]  / masse_view[massen_index]
    result[2] = kraft_view[2]  / masse_view[massen_index]

Overwriting /tmp/cython_galaxy/beschleunigung.pyx


In [96]:
%%file /tmp/cython_galaxy/positions_aktualliersierung.pyx
cimport cython

#Positions Aktualliersierung
#position = position + dt * geschwindigkeit  + ((dt ** 2) / 2) * Beschleunigung_Erde
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Positions_aktualliersierung (double [:, :] position_view , double [:, :] geschwindigkeit_view, \
                   double [:] beschleunigung_view, double dt, int planet_index) nogil:
    
    position_view[planet_index][0] = position_view[planet_index][0]  + dt * geschwindigkeit_view[planet_index][0] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[0]
        
    position_view[planet_index][1] = position_view[planet_index][1]  + dt * geschwindigkeit_view[planet_index][1] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[1]
        
    position_view[planet_index][2] = position_view[planet_index][2]  + dt * geschwindigkeit_view[planet_index][2] \
                                                + ((dt ** 2) / 2) * beschleunigung_view[2]

Overwriting /tmp/cython_galaxy/positions_aktualliersierung.pyx


In [97]:
%%file /tmp/cython_galaxy/geschwindigkeit_aktualliersierung.pyx
cimport cython

#Geschwindigkeits Aktuallisierung
#geschwindigkeit[0][0]  += dt * Beschleunigung_Erde[0]
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Geschwindigkeit_aktualliersierung (double [:, :] geschwindigkeit_view , double [:] beschleunigung_view, \
                   double dt ,int planet_index) nogil:
    geschwindigkeit_view[planet_index][0] = geschwindigkeit_view[planet_index][0] + dt * beschleunigung_view[0]
    geschwindigkeit_view[planet_index][1] = geschwindigkeit_view[planet_index][1] + dt * beschleunigung_view[1]
    geschwindigkeit_view[planet_index][2] = geschwindigkeit_view[planet_index][2] + dt * beschleunigung_view[2]

Overwriting /tmp/cython_galaxy/geschwindigkeit_aktualliersierung.pyx


## Gil Version Finished and Debuged

In [137]:
%%file /tmp/cython_galaxy/loop.pyx
from cython.parallel import prange
from abstand import Abstand
from betrag import Betrag
from kraft import Kraft
from beschleunigung import Beschleunigung
from positions_aktualliersierung import Positions_aktualliersierung
from geschwindigkeit_aktualliersierung import Geschwindigkeit_aktualliersierung
import numpy as np
cimport cython

def Loop (pos_list):
    #Erstellen der Python Listen
    python_position = [[149_597_890_000, 0,0], [0,0,0]]
    python_geschwindigkeit = [[0, 29.786 * 10**3,0], [0,0,0]]
    python_masse = [5.97219 * 10 ** 24, 1.9889 * 10 ** 30]
    python_position_neu = []
    python_geschwindigkeit_neu = []
    python_position_list = []

    #Umwandeln in Numpy Arrays
    position = np.array(python_position, dtype=np.float64)
    geschwindigkeit = np.array(python_geschwindigkeit,dtype=np.float64)
    masse = np.array(python_masse,dtype=np.float64)
    position_neu = np.array(python_position_neu ,dtype=np.float64)
    geschwindigkeit_neu = np.ndarray(python_geschwindigkeit_neu ,dtype=np.float64)
    
    #Umwandlung in MemoryView
    cdef double [:, :] position_view = position
    cdef double [:, :] geschwindigkeit_view = geschwindigkeit
    cdef double [:] masse_view = masse
    
    #Statische Variable
    cdef int dt = 60
    cdef long Jahr = 360 * 24 * 60 * 2
    cdef int i
    cdef int debug = 1
    
    #Temporaere MemoryView zum ZwischenSpeichern
    cdef double [:] Abstand_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Abstand_Sonne = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Betrag_Erde = np.array([0],  dtype=np.float64)
    cdef double [:] Betrag_Sonne = np.array([0],  dtype=np.float64) 
    cdef double [:] Kraft_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Kraft_Sonne = np.array([0,0,0],  dtype=np.float64)  
    cdef double [:] Beschleunigung_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Beschleunigung_Sonne = np.array([0,0,0],  dtype=np.float64)
    
    cpdef int Erde_index = 0
    cpdef int Sonne_index = 1

    #for i in prange (Jahr, nogil=True): 
    for i in range (Jahr): 

        ## Ausrechnen
        # Erde = 0, Sonne = 1
        if (i > 1 and i <20):
            print("position_view[Erde_index]", position_view[Erde_index][0],\
                 position_view[Erde_index][1], position_view[Erde_index][2])
        
        Abstand(position_view, Abstand_Erde, Erde_index, Sonne_index)
        Abstand(position_view, Abstand_Sonne, Sonne_index, Erde_index)
        
        if (i == 0 and debug):
            print("Abstand", Abstand_Erde[0], Abstand_Erde[1], Abstand_Erde[2])
  
        Betrag(Abstand_Erde, Betrag_Erde)
        Betrag(Abstand_Sonne, Betrag_Sonne)
        
        if (i == 0 and debug):
            print("Betrag", Betrag_Erde[0])

        Kraft(masse_view, Abstand_Erde, Betrag_Erde, Erde_index, Sonne_index, Kraft_Erde)
        Kraft(masse_view, Abstand_Sonne, Betrag_Sonne, Erde_index, Sonne_index, Kraft_Sonne)
        
        if (i == 0 and debug):

            print("Kraft", Kraft_Erde[0],Kraft_Erde[1], Kraft_Erde[2])

        Beschleunigung(Kraft_Erde, masse_view, Erde_index, Beschleunigung_Erde)
        Beschleunigung(Kraft_Sonne, masse_view, Sonne_index, Beschleunigung_Sonne)
        
        if (i == 0 and debug):
            print("Beschleunigung", Beschleunigung_Erde[0], \
                 Beschleunigung_Erde[1], Beschleunigung_Erde[2])

        ## Aktualisieren
        if (i == 0 and debug):
            print("position_view_first", position_view[0][0], \
                    position_view[0][1], position_view[0][2])
        
        Positions_aktualliersierung(position_view, geschwindigkeit_view, Beschleunigung_Erde,\
                                     dt, Erde_index)
        Geschwindigkeit_aktualliersierung (geschwindigkeit_view ,Beschleunigung_Erde, \
                       dt ,Erde_index)
        if (i == 0 and debug):
            print("position_view", position_view[0][0], 
                  position_view[0][1], position_view[0][2])
        
        Positions_aktualliersierung(position_view, geschwindigkeit_view, Beschleunigung_Sonne,\
                                     dt, Sonne_index)
        Geschwindigkeit_aktualliersierung (geschwindigkeit_view ,Beschleunigung_Sonne, \
                       dt ,Sonne_index)
        if (i == 0 and debug):
            print("geschwindigkeit_view", geschwindigkeit_view[0][0],
                 geschwindigkeit_view[0][1], geschwindigkeit_view[0][2])

        pos_list.append(np.array((position_view[Erde_index][0], position_view[Erde_index][1],\
                 position_view[Erde_index][2]), np.float32))

Overwriting /tmp/cython_galaxy/loop.pyx


In [138]:
%%file /tmp/cython_galaxy/setup.py


# Aufruf: python3 setup.py build_ext --inplace
# Windows: zusaetzliche Option --compiler=mingw32
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy

ext_modules=[ Extension("abstand", ["abstand.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
             Extension("kraft", ["kraft.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
              Extension("betrag", ["betrag.pyx"],
        #extra_compile_args=['-O3'], libraries=['m'],
        extra_compile_args=['-O3'], libraries=['m']),
            #include_dirs=[numpy.get_include()]),
             Extension("beschleunigung", ["beschleunigung.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
             Extension("positions_aktualliersierung", ["positions_aktualliersierung.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
             Extension("geschwindigkeit_aktualliersierung", ["geschwindigkeit_aktualliersierung.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
             Extension("loop", ["loop.pyx"],
        extra_compile_args=['-O3'], libraries=['m'])
]
             
setup( name = 'cython demo',
  cmdclass = {'build_ext': build_ext},
  ext_modules = ext_modules)



Overwriting /tmp/cython_galaxy/setup.py


In [139]:
%%bash
cd /tmp/cython_galaxy
python3 setup.py build_ext --inplace

running build_ext
skipping 'abstand.c' Cython extension (up-to-date)
skipping 'kraft.c' Cython extension (up-to-date)
skipping 'betrag.c' Cython extension (up-to-date)
skipping 'beschleunigung.c' Cython extension (up-to-date)
skipping 'positions_aktualliersierung.c' Cython extension (up-to-date)
skipping 'geschwindigkeit_aktualliersierung.c' Cython extension (up-to-date)
cythoning loop.pyx to loop.c
building 'loop' extension
gcc -pthread -B /opt/anaconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/anaconda/include/python3.6m -c loop.c -o build/temp.linux-x86_64-3.6/loop.o -O3
gcc -pthread -shared -B /opt/anaconda/compiler_compat -L/opt/anaconda/lib -Wl,-rpath=/opt/anaconda/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.6/loop.o -L/opt/anaconda/lib -lm -lpython3.6m -o /tmp/cython_galaxy/loop.cpython-36m-x86_64-linux-gnu.so


In [140]:
%%file /tmp/cython_galaxy/start.py
from loop import Loop
import matplotlib.pyplot as plt

python_position_list = []
Loop(python_position_list)
plt.plot([x[0] for x in python_position_list], [x[1] for x in python_position_list])
plt.show()

for _ in range (20):
    print(python_position_list[_])

Overwriting /tmp/cython_galaxy/start.py


In [141]:
%%bash
cd /tmp/cython_galaxy
python3 start.py

('Abstand', -149597890000.0, 0.0, 0.0)
('Betrag', 149597890000.0)
('Kraft', -3.541209863507121e+22, 0.0, 0.0)
('Beschleunigung', -0.005929499670149678, 0.0, 0.0)
('position_view_first', 149597890000.0, 0.0, 0.0)
('position_view', 149597889989.3269, 1787160.0, 0.0)
('geschwindigkeit_view', -0.3557699802089807, 29786.0, 0.0)
('position_view[Erde_index]', 149597889957.30762, 3574319.9998724945, 0.0)
('position_view[Erde_index]', 149597889903.94214, 5361479.999362472, 0.0)
('position_view[Erde_index]', 149597889829.23044, 7148639.998214924, 0.0)
('position_view[Erde_index]', 149597889733.17255, 8935799.996174837, 0.0)
('position_view[Erde_index]', 149597889615.76846, 10722959.9929872, 0.0)
('position_view[Erde_index]', 149597889477.0182, 12510119.988397006, 0.0)
('position_view[Erde_index]', 149597889316.9217, 14297279.982149241, 0.0)
('position_view[Erde_index]', 149597889135.479, 16084439.973988898, 0.0)
('position_view[Erde_index]', 149597888932.69012, 17871599.96366096, 0.0)
('position

In [ ]:
#Vergleich Werte vom alten Prototypen
Abstand_Erde [ -1.49597890e+11   0.00000000e+00]
Betrag_Erde 149597890000.0
Kraft_Erde [ -3.54120986e+22   0.00000000e+00]
Beschleunigung_Erde [-0.0059295  0.       ]
Position_Erde [  1.49597890e+11   1.78716000e+06]
Geschwindigkeit_Erde [ -3.55769980e-01   2.97860000e+04]

#Neue Werte von cython_galaxy
Abstand -149597890000.0 0.0 0.0
Betrag 149597890000.0
Kraft -3.541209863507121e+22 0.0 0.0
Beschleunigung -0.005929499670149678 0.0 0.0
position_view 149597889989.3269 1787160.0 0.0
geschwindigkeit_view -0.3557699802089807 29786.0 0.0

## NoGil Test

In [223]:
%%file /tmp/cython_galaxy/loop.pyx
from cython.parallel import prange
from abstand import Abstand
from betrag import Betrag
from kraft import Kraft
from beschleunigung import Beschleunigung
from positions_aktualliersierung import Positions_aktualliersierung
from geschwindigkeit_aktualliersierung import Geschwindigkeit_aktualliersierung
import numpy as np
cimport cython


def Loop (pos_list):
    #Erstellen der Python Listen
    python_position = [[149_597_890_000, 0,0], [0,0,0]]
    python_geschwindigkeit = [[0, 29.786 * 10**3,0], [0,0,0]]
    python_masse = [5.97219 * 10 ** 24, 1.9889 * 10 ** 30]
    python_position_neu = []
    python_geschwindigkeit_neu = []
    python_position_list = []

    #Umwandeln in Numpy Arrays
    position = np.array(python_position, dtype=np.float64)
    geschwindigkeit = np.array(python_geschwindigkeit,dtype=np.float64)
    masse = np.array(python_masse,dtype=np.float64)
    position_neu = np.array(python_position_neu ,dtype=np.float64)
    geschwindigkeit_neu = np.ndarray(python_geschwindigkeit_neu ,dtype=np.float64)
    
    #Umwandlung in MemoryView
    cdef double [:, :] position_view = position
    cdef double [:, :] geschwindigkeit_view = geschwindigkeit
    cdef double [:] masse_view = masse
    
    #Statische Variable
    cdef int dt = 60
    cdef long Jahr = 360 * 24 * 60 * 2
    cdef int i
    cdef int debug = 0
    
    #Temporaere MemoryView zum ZwischenSpeichern
    cdef double [:] Abstand_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Abstand_Sonne = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Betrag_Erde = np.array([0],  dtype=np.float64)
    cdef double [:] Betrag_Sonne = np.array([0],  dtype=np.float64) 
    cdef double [:] Kraft_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Kraft_Sonne = np.array([0,0,0],  dtype=np.float64)  
    cdef double [:] Beschleunigung_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Beschleunigung_Sonne = np.array([0,0,0],  dtype=np.float64)
    
    cpdef int Erde_index = 0
    cpdef int Sonne_index = 1
    

        
    for i in prange (Jahr, nogil=True): 
    #for i in range (Jahr): 

        ## Ausrechnen
        # Erde = 0, Sonne = 1
        Abstand(position_view, Abstand_Erde, Erde_index, Sonne_index)
        Abstand(position_view, Abstand_Sonne, Sonne_index, Erde_index)

        Betrag(Abstand_Erde, Betrag_Erde)
        Betrag(Abstand_Sonne, Betrag_Sonne)

        Kraft(masse_view, Abstand_Erde, Betrag_Erde, Erde_index, Sonne_index, Kraft_Erde)
        Kraft(masse_view, Abstand_Sonne, Betrag_Sonne, Erde_index, Sonne_index, Kraft_Sonne)

        Beschleunigung(Kraft_Erde, masse_view, Erde_index, Beschleunigung_Erde)
        Beschleunigung(Kraft_Sonne, masse_view, Sonne_index, Beschleunigung_Sonne)

        ## Aktualisieren  
        Positions_aktualliersierung(position_view, geschwindigkeit_view, Beschleunigung_Erde,\
                                     dt, Erde_index)
        Geschwindigkeit_aktualliersierung (geschwindigkeit_view ,Beschleunigung_Erde, \
                       dt ,Erde_index)
        Positions_aktualliersierung(position_view, geschwindigkeit_view, Beschleunigung_Sonne,\
                                     dt, Sonne_index)
        Geschwindigkeit_aktualliersierung (geschwindigkeit_view ,Beschleunigung_Sonne, \
                       dt ,Sonne_index)
        with gil:
            pos_list.append(np.array((position_view[Erde_index][0], position_view[Erde_index][1],\
                 position_view[Erde_index][2]), np.float32))

Overwriting /tmp/cython_galaxy/loop.pyx


In [224]:
%%bash
cd /tmp/cython_galaxy
python3 setup.py build_ext --inplace

running build_ext
skipping 'abstand.c' Cython extension (up-to-date)
skipping 'kraft.c' Cython extension (up-to-date)
skipping 'betrag.c' Cython extension (up-to-date)
skipping 'beschleunigung.c' Cython extension (up-to-date)
skipping 'positions_aktualliersierung.c' Cython extension (up-to-date)
skipping 'geschwindigkeit_aktualliersierung.c' Cython extension (up-to-date)
cythoning loop.pyx to loop.c
building 'loop' extension
gcc -pthread -B /opt/anaconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/anaconda/include/python3.6m -c loop.c -o build/temp.linux-x86_64-3.6/loop.o -O3



Error compiling Cython file:
------------------------------------------------------------
...
    for i in prange (Jahr, nogil=True): 
    #for i in range (Jahr): 

        ## Ausrechnen
        # Erde = 0, Sonne = 1
        Abstand(position_view, Abstand_Erde, Erde_index, Sonne_index)
              ^
------------------------------------------------------------

loop.pyx:59:15: Discarding owned Python object not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
    for i in prange (Jahr, nogil=True): 
    #for i in range (Jahr): 

        ## Ausrechnen
        # Erde = 0, Sonne = 1
        Abstand(position_view, Abstand_Erde, Erde_index, Sonne_index)
              ^
------------------------------------------------------------

loop.pyx:59:15: Calling gil-requiring function not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
    for i in prange (Jahr, nogi

In [216]:
%%bash
cd /tmp/cython_galaxy
python3 start.py

Traceback (most recent call last):
  File "start.py", line 1, in <module>
    from loop import Loop
ImportError: /tmp/cython_galaxy/loop.cpython-36m-x86_64-linux-gnu.so: undefined symbol: Beschleunigung


## Short NoGIL Test

In [172]:
%%file /tmp/cython_galaxy/abstand.pyx
cimport cython
#Abstands Funktion
#Abstand_Sonne = position[1] - position[0]

@cython.final
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (double [:, :] position_view , double[:] result ,int first, int second) nogil:   
    result[0] = position_view[second][0] - position_view[first][0]
    result[1] = position_view[second][1] - position_view[first][1]
    result[2] = position_view[second][2] - position_view[first][2]

Overwriting /tmp/cython_galaxy/abstand.pyx


In [206]:
%%file /tmp/cython_galaxy/loop.pyx
from cython.parallel import prange
from abstand import Abstand
from betrag import Betrag
from kraft import Kraft
from beschleunigung import Beschleunigung
from positions_aktualliersierung import Positions_aktualliersierung
from geschwindigkeit_aktualliersierung import Geschwindigkeit_aktualliersierung
import numpy as np
cimport cython




def Loop (pos_list):
    #Erstellen der Python Listen
    python_position = [[149_597_890_000, 0,0], [0,0,0]]
    python_geschwindigkeit = [[0, 29.786 * 10**3,0], [0,0,0]]

    #Umwandeln in Numpy Arrays
    position = np.array(python_position, dtype=np.float64)
    geschwindigkeit = np.array(python_geschwindigkeit,dtype=np.float64)
    
    #Umwandlung in MemoryView
    cdef double [:, :] position_view = position
    cdef double [:, :] geschwindigkeit_view = geschwindigkeit
    
    #Statische Variable
    cdef int dt = 60
    cdef long Jahr = 360 * 24 * 60 * 2
    cdef int i
    cdef int debug = 0
    
    #Temporaere MemoryView zum ZwischenSpeichern
    cdef double [:] Abstand_Erde = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Abstand_Sonne = np.array([0,0,0],  dtype=np.float64)
    cdef double [:] Betrag_Erde = np.array([0],  dtype=np.float64)
    cdef double [:] Betrag_Sonne = np.array([0],  dtype=np.float64) 
    
    cpdef int Erde_index = 0
    cpdef int Sonne_index = 1
    
    cdef extern void Abstand(double [:, :] position_view , double[:] result ,int first, int second) nogil

    for i in prange (Jahr, nogil=True): 
        
            Abstand(position_view, Abstand_Erde, Erde_index, Sonne_index)
            Abstand(position_view, Abstand_Sonne, Sonne_index, Erde_index)

Overwriting /tmp/cython_galaxy/loop.pyx


In [207]:
%%bash
cd /tmp/cython_galaxy
python3 setup.py build_ext --inplace

running build_ext
skipping 'abstand.c' Cython extension (up-to-date)
skipping 'kraft.c' Cython extension (up-to-date)
skipping 'betrag.c' Cython extension (up-to-date)
skipping 'beschleunigung.c' Cython extension (up-to-date)
skipping 'positions_aktualliersierung.c' Cython extension (up-to-date)
skipping 'geschwindigkeit_aktualliersierung.c' Cython extension (up-to-date)
cythoning loop.pyx to loop.c
building 'loop' extension
gcc -pthread -B /opt/anaconda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/anaconda/include/python3.6m -c loop.c -o build/temp.linux-x86_64-3.6/loop.o -O3
gcc -pthread -shared -B /opt/anaconda/compiler_compat -L/opt/anaconda/lib -Wl,-rpath=/opt/anaconda/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.6/loop.o -L/opt/anaconda/lib -lm -lpython3.6m -o /tmp/cython_galaxy/loop.cpython-36m-x86_64-linux-gnu.so


loop.c: In Funktion »__pyx_pf_4loop_Loop«:
loop.c:2335:29: Warnung: Implizite Deklaration der Funktion »Abstand«; meinten Sie »srand«? [-Wimplicit-function-declaration]
                             Abstand(__pyx_v_position_view, __pyx_v_Abstand_Erde, __pyx_v_Erde_index, __pyx_v_Sonne_index);
                             ^~~~~~~
                             srand


In [184]:
%%bash
cd /tmp/cython_galaxy
python3 start.py

Traceback (most recent call last):
  File "start.py", line 10, in <module>
    print(python_position_list[_])
IndexError: list index out of range


## Cython Test

## Versuch vom 30.11.2017



python listen von zb geschwindigkeit...

umwandeln in numpy.array oder numpy.ndarray ?

übergeben an die cython funktion (nogil)

zugriff innerhalb der cython funktion (memoryview?)



## Erster Numpy Test mit Listen als numpy Array

In [ ]:
import numpy as np
planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)

print (position)
print()
print(position[0][0])


## Memory View Beispiel

In [ ]:
%load_ext Cython

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np

narr = np.arange(9, dtype=np.dtype("d")).reshape((3, 3))
cdef double [:, :] narr_view = narr

print (narr)
print (narr_view)
print (narr_view[2][2])

## Memory View Beispiel Test Umbau

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np

planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)
cdef float [:, :] position_view = position
print (position)
print (position_view)
print (position_view[0][0])
print (position_view[2][2])

position_view[2][2] = 5
print (position_view[2][2])

## Erfolgreicher NoGil Test mit Memory View

In [ ]:
%%cython -f -c-fopenmp --link-args=-fopenmp

import numpy as np
cimport numpy as cnp
cimport cython
ctypedef cnp.float32_t dtype_t

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
position = np.array(planet_position,  dtype=np.float32)
cdef float [:, :] position_view = position
cdef float [:] result1 = np.array([0,0,0],  dtype=np.float32)
cdef float [:] result2 = np.array([0,0,0],  dtype=np.float32)

Abstand(position_view, result1, 0, 1)
position_view_result1 = np.asarray(result1)

Abstand(position_view, result2, 1, 0)
position_view_result2 = np.asarray(result2)

print (position_view_result1)
print (position_view_result2)

## Build Test

In [ ]:
import os
os.mkdir('/tmp/cython_galaxy_einzel_build')

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/abstand.pyx
#Abstands Funktion
#Abstand_Sonne[0] = position[1][0] - position[0][0]
cimport cython

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef void Abstand (float [:, :] position_view , float[:] result ,int first, int second) nogil:   
    result[0] = position_view[first][0] - position_view[second][0]
    result[1] = position_view[first][1] - position_view[second][1]
    result[2] = position_view[first][2] - position_view[second][2]

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/abstand_test.pyx

import numpy as np
from abstand import Abstand 
cimport numpy as cnp
cimport cython
ctypedef cnp.float32_t dtype_t

def test():
    planet_position = [[149_597_890_000, 0,0], [0,0,0], [1,1,1]]
    position = np.array(planet_position,  dtype=np.float32)
    cdef float [:, :] position_view = position
    cdef float [:] result1 = np.array([0,0,0],  dtype=np.float32)
    cdef float [:] result2 = np.array([0,0,0],  dtype=np.float32)

    Abstand(position_view, result1, 0, 1)
    position_view_result1 = np.asarray(result1)

    Abstand(position_view, result2, 1, 0)
    position_view_result2 = np.asarray(result2)

    print (position_view_result1)
    print (position_view_result2)

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/start.py

from abstand_test import test
test()

In [ ]:
%%file /tmp/cython_galaxy_einzel_build/setup.py


# Aufruf: python3 setup.py build_ext --inplace
# Windows: zusaetzliche Option --compiler=mingw32
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy

ext_modules=[ Extension("abstand", ["abstand.pyx"],
        extra_compile_args=['-O3'], libraries=['m']),
              Extension("abstand_test", ["abstand_test.pyx"],
        extra_compile_args=['-O3'], libraries=['m'],
            include_dirs=[numpy.get_include()])
            ]
             
setup( name = 'cython demo',
  cmdclass = {'build_ext': build_ext},
  ext_modules = ext_modules)


## Build Script

In [ ]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 setup.py build_ext --inplace

In [ ]:
%%bash
cd /tmp/cython_galaxy_einzel_build
python3 start.py